In [ ]:
import psycopg2
import geopandas as gpd
import os
import sys
import glob
import datetime

# adds the package path to the Python path to make sure all the local imports work fine 
if os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))) not in sys.path:
    sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))
    
from wp4.constants import DB_HOST, DB_NAME, DB_USER, DB_PASS, DATA_DIR_FIRES

# Initiate connection to spatial database (PostgreSQL/PostGIS)

In [ ]:
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASS, host=DB_HOST)
cur = conn.cursor()

In [ ]:
# File name of the dataset, so everything before .shp
dataset_name = ""  

# Column name/string to use as reference. Eg: Blackstairs Mountains Fire Brigade
reference = "" 

# String indicating the source of the fire event report - Currently used: Fire Brigade, Satellite & Media
source = "" 

# Name of the column containing the location information, if none available use None
location = "" 

# Name of the column containing detailed information about the fire event, if none available use None
info = "" 

datetime = {
    'DATE':"", # Name of the column containing date information
    # 'TIME':"", # Name of the column containing time information if available, please uncomment this line 
    'PATTERN':"", # Pattern to obtain a datetime object from the date and or time string, more info:
                  # https://www.programiz.com/python-programming/datetime/strptime 
}

# Name of the column containing information of the fire type if available, if none available use None
fire_type = "" 

# Dict containing information on which label to assign to which. Labels currently in use are:
# Bog Fire, Bush Fire, Grass Fire, Forest Fire, Gorse Fire, Controlled Fire
fire_type_label_map = {  
                         
    'dataset_value':'label'
}

# Name of the column containing the FRP value, if none available use None
FRP = ""


column_info = {
    'DATETIME':{ # Column names containing the Date and/or Time information
        dataset_name:datetime,
    },
    'SOURCE':{  # To use as property indicating the general source of the data (Fire Brigade, Satellite or Media)
        dataset_name:source,
    },
    'LOCATION':{ # Column containing the location information
        dataset_name:location,
    },
    'REFERENCE':{  # Reference to the specific source of information
        dataset_name:reference,
    },
    'INFO':{  # In case detailed information about the fire is available 
        dataset_name:info,
    },
    'TYPE':{ # Collection of dictionaries to use to reassign labels specifying the type of Fire
        # The following labels been used: Bog Fire, Bush Fire, Grass Fire, Forest Fire, Gorse Fire, Controlled Fire
        dataset_name:{
            fire_type:fire_type_label_map,
        }
    },
    'FRP':{  # In case detailed information about the fire is available 
        dataset_name:FRP,
    }
}

In [ ]:
# Load all the column information for the specific dataset    
time_info = column_info['DATETIME'][dataset_name]
label_info = column_info['TYPE'][dataset_name] 
source_info = column_info['SOURCE'][dataset_name]
location_info = column_info['LOCATION'][dataset_name]
reference_info = column_info['REFERENCE'][dataset_name]
info_info = column_info['INFO'][dataset_name]
frp_info = column_info['FRP'][dataset_name]

# Load the shapefile, drop rows with null values for the geometry/date column and reproject to WGS 84
dataset = gpd.read_file(glob.glob(f"{DATA_DIR_FIRES}/{dataset_name}/*.shp")[0])
dataset = dataset.dropna(subset=['geometry', time_info['DATE']]).to_crs('EPSG:4326').copy()

def convert_to_datetime(x):
    """Function to convert the date and/or time information into a datetime object"""

    if 'TIME' in time_info.keys():
        # For some datasets the time column was not consistently formatted
        # as a result some try/except blocks were needed
        try:
            return datetime.datetime.strptime(
                f"{x[time_info['DATE']]}{x[time_info['TIME']]}",
                time_info['PATTERN'])
        except ValueError:
            try:
                return datetime.datetime.strptime(
                    f"{x[time_info['DATE']]}{int(x[time_info['TIME']])}",
                    time_info['PATTERN']
                )
            except:
                return datetime.datetime.strptime(
                    f"{x[time_info['DATE']]}{x[time_info['TIME']]}",
                    '%Y-%m-%d%H:%M:%S'
                )

    else:
        return datetime.datetime.strptime(
            f"{x[time_info['DATE']]}",
            time_info['PATTERN']
        )
    



def assign_labels(x):
    """Function to give fire types from different databases consistent naming"""
    try:
        return label_info[list(label_info.keys())[0]][x]
    except:
        return None


dataset['DATETIME'] = dataset.apply(convert_to_datetime, 1)

# Add columns to the dataset containing the information that will be inserted into the database
if label_info is not None:
    dataset['TYPE'] = dataset[list(label_info.keys())[0]].apply(assign_labels)
else:
    dataset['TYPE'] = None
if source_info is not None:
    dataset['SOURCE'] = source_info
else:
    dataset['SOURCE'] = None
if location_info is not None:
    dataset['LOCATION'] = dataset[location_info]
else:
    dataset['LOCATION'] = None
if reference_info is not None:
    try:
        dataset['REFERENCE'] = dataset[reference_info]
    except:
        dataset['REFERENCE'] = reference_info
else:
    dataset['REFERENCE'] = None
if info_info is not None:
    dataset['INFO'] = dataset[info_info]
else:
    dataset['INFO'] = None

if frp_info is not None:
    dataset['FRP'] = dataset[frp_info]
else:
    dataset['FRP']  = None

# Loop through all the fires in the dataset and insert them into the database
for ind, row in dataset.iterrows():
    cur.execute(
        """INSERT INTO public.all_fire_events(
        DATETIME, GEOMETRY, SOURCE, LOCATION, REFERENCE, TYPE, INFO) VALUES(%s, ST_SetSRID(ST_MakePoint(%s, %s),4326), %s, %s, %s, %s, %s)""",
        (row['DATETIME'],
         row['geometry'].x,
         row['geometry'].y,
         row['SOURCE'],
         row['LOCATION'],
         row['REFERENCE'],
         row['TYPE'],
         row['INFO'],
        row['FRP'])
    )
    conn.commit()